In [1]:
import numpy as np
import os
import cv2
import numpy as np
import librosa
from pydub import AudioSegment

C:\Users\jbzuc\Anaconda3\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
# Get voice filename list
voice_filenames = os.listdir(path='voices/')
# Get Face filename list
face_filenames = os.listdir(path='faces/')

SAMPLE_LENGTH = 4000      #Works in milliseconds

In [10]:
n = 0 # initialize ID = 0
ID_code_mapping = []
for f_file in face_filenames:
    code6, _ = f_file.split('.') # get 6-digit participant code
    code6 = code6[len(code6)-6:] # double check that this works
    v_file = get_voice_file(code6) # try and find a corresponding voice file

    if v_file:
        print(n)
        
        # save spectrograms
        spectrogram_list = generate_spectrogram('voices/' + v_file, n)
        for m, spectrogram in enumerate(spectrogram_list):
            np.savetxt("voicespecs/voice_{}_{}".format(n, m), spectrogram, delimiter=',')
        
        # save image
        image_matrix = np.float64(cv2.imread('faces/'+f_file,-1))
        np.savetxt("facespecs/face_{}".format(n), image_matrix, delimiter=',')
        
        # add code to the ID-code mapping list
        ID_code_mapping.append(code6)
        n += 1


0
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
1
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)
(1025, 251)


KeyboardInterrupt: 

In [6]:
ID = np.array(ID_code_mapping)
#print(ID_code_mapping)
np.savetxt("map.csv", ID_code_mapping, delimiter=',', fmt='%s')

['140012', '140017', '140018', '140022', '140023', '140025', '140027', '140028', '140029', '140030', '140031', '140032', '140033', '140034', '140035', '140036', '140037', '140038', '140039', '140040', '140041', '140042', '140043', '140044', '140045', '140046', '140047', '140048', '140049', '140050', '140051', '140052', '140053', '140054', '140055', '140056', '140057', '140058', '140059', '140060', '140061', '140062', '140063', '140064', '140065', '140066', '140067', '140068', '140069', '140070', '140071', '140072', '140073', '140074', '140075', '140076', '140077', '140078', '140079', '140080', '140081', '140082', '140083', '140084', '140085', '140086', '140087', '140088', '140089', '140090', '140091', '140092', '140093', '140094', '140095', '140096', '140097', '140098', '140099', '140100', '140101', '140102', '140103', '140105', '140107', '140108', '140109', '140110', '140111', '140112', '140113', '140114', '140115', '140116', '140117', '140118', '140119', '140120', '140121', '140122',

In [7]:
def get_voice_file(code6):
    """
    Given a 6 digit string, return the first voice filename that contains that string.
    Requires global variable voice_file_list.
    """
    for v_file in voice_filenames:
        if v_file.find(code6) >= 0:
            return v_file
    return None

In [8]:
def generate_spectrogram(voice_file_path, n):
    skip_duration = 0 #Works in milliseconds
    spectrogram_array = []
    newAudio = AudioSegment.from_wav(voice_file_path)
    m = 0 #Segment counter
    while((skip_duration/1000) < (newAudio.duration_seconds - SAMPLE_LENGTH/1000)):
       
        newAudio1 = newAudio[skip_duration : SAMPLE_LENGTH + skip_duration]
        new_path = 'voiceclips/'+'voice_'+str(n)+'_'+str(m)+'.wav'
        newAudio1.export(new_path, format="wav")
        # Save audio segments to new path
        audio,sr = librosa.load(new_path, sr = 16000)
        spectrogram = librosa.stft(audio, n_fft = 2048, hop_length = 256, win_length = 2048)
        speech = abs(spectrogram)
        phase = spectrogram / (speech  + 2.2204e-16)
        # Save stft spectrograms
        spectrogram_array.append(speech)
        #np.savetxt('phase/phase'+str(n)+'_'+str(m)+'.csv',phase,delimiter=',')
        skip_duration+=2000
        m += 1
    return spectrogram_array